In [31]:
#pip install pycoingecko
#pip install yahooquery
#import modules
import pandas as pd
import datetime
import time as t
import plotly.graph_objects as go
from plotly.offline import plot
from pycoingecko import CoinGeckoAPI
import yahooquery as yq
from yahooquery import Ticker


CRYPTO

In [32]:
#create a client
cg = CoinGeckoAPI()
#confirm connection
CoinGeckoAPI().ping()

{'gecko_says': '(V3) To the Moon!'}

In [33]:
#get a list of coins, sort df by id
coin_list = CoinGeckoAPI().get_coins_list()
coin_df = pd.DataFrame(coin_list).sort_values('symbol').reset_index(drop=True)
coin_df.head()

,id,symbol,name
0,arabian-doge,$adoge,Arabian Doge
1,alkimi,$ads,Alkimi
2,aurora-token,$adtx,Aurora Dimension
3,alpha-nodes,$alpha,Alpha Shares
4,alpha-labz,$alpha,Alpha Labz


In [34]:
coins = ['bitcoin','ethereum','bnb','binance','xrp','cardano','solana','dodgecoin','polkadot']
coin_df.loc[(coin_df['id']=='bitcoin')|(coin_df['id']=='ethereum')|
            (coin_df['id']=='bnb')|(coin_df['id']=='ripple')|
            (coin_df['id']=='cardano')|(coin_df['id']=='solana')|
            (coin_df['id']=='doge')|(coin_df['id']=='polkadot'),:]
# coin_df.loc[(coin_df['symbol']=='btc')|(coin_df['symbol']=='eth')|
#             (coin_df['symbol']=='bnb')|(coin_df['symbol']=='xrp')|
#             (coin_df['symbol']=='ada')|(coin_df['symbol']=='sol')|
#             (coin_df['symbol']=='doge')|(coin_df['symbol']=='dot')
#             |(coin_df['symbol']=='binance'),:]

,id,symbol,name
308,cardano,ada,Cardano
1880,bitcoin,btc,Bitcoin
3516,polkadot,dot,Polkadot
4039,ethereum,eth,Ethereum
10637,solana,sol,Solana
12858,ripple,xrp,XRP


In [36]:
counterCurrencies = CoinGeckoAPI().get_supported_vs_currencies()
counterCurrencies
currency = ['usd']
simplePriceRequest = CoinGeckoAPI().get_price(ids = coins, vs_currencies = 'usd')
simplePriceRequest

{'polkadot': {'usd': 5.87},
 'ethereum': {'usd': 1349.39},
 'solana': {'usd': 28.62},
 'bitcoin': {'usd': 19392.25},
 'cardano': {'usd': 0.356515}}

In [37]:
marketcap_vol_change = CoinGeckoAPI().get_price(ids = coins, 
                        vs_currencies = currency, 
                        include_market_cap = True,
                        include_24hr_vol = True,
                        include_24hr_change = True,
                        include_last_updated_at = True)
print(marketcap_vol_change)

{'polkadot': {'usd': 5.87, 'usd_market_cap': 6836980256.361288, 'usd_24h_vol': 227617028.2702934, 'usd_24h_change': 0.13001634269591586, 'last_updated_at': 1666583258}, 'ethereum': {'usd': 1349.23, 'usd_market_cap': 162595858967.03848, 'usd_24h_vol': 8845903947.22309, 'usd_24h_change': 2.764292706078031, 'last_updated_at': 1666583247}, 'solana': {'usd': 28.62, 'usd_market_cap': 10256236232.39152, 'usd_24h_vol': 703226376.7576417, 'usd_24h_change': 0.7520341086838184, 'last_updated_at': 1666583249}, 'bitcoin': {'usd': 19391.05, 'usd_market_cap': 372036670113.2621, 'usd_24h_vol': 22870141203.423405, 'usd_24h_change': 1.0090360456218648, 'last_updated_at': 1666583238}, 'cardano': {'usd': 0.356585, 'usd_market_cap': 12497116368.331413, 'usd_24h_vol': 372020783.6923372, 'usd_24h_change': 0.5219797985565214, 'last_updated_at': 1666583258}}


In [42]:
#dataframe and transposed
marketcap_vol_change_df=pd.DataFrame(marketcap_vol_change).T
marketcap_vol_change_df

,usd,usd_market_cap,usd_24h_vol,usd_24h_change,last_updated_at
polkadot,5.870000,6.836980e+09,2.276170e+08,0.130016,1.666583e+09
ethereum,1349.230000,1.625959e+11,8.845904e+09,2.764293,1.666583e+09
solana,28.620000,1.025624e+10,7.032264e+08,0.752034,1.666583e+09
bitcoin,19391.050000,3.720367e+11,2.287014e+10,1.009036,1.666583e+09
cardano,0.356585,1.249712e+10,3.720208e+08,0.521980,1.666583e+09


In [45]:
# #reformat date
marketcap_vol_change_df['last_updated_at'] = pd.to_datetime(marketcap_vol_change_df['last_updated_at'],unit='s')
print(marketcap_vol_change_df['last_updated_at'])

polkadot   2022-10-24 03:47:38
ethereum   2022-10-24 03:47:27
solana     2022-10-24 03:47:29
bitcoin    2022-10-24 03:47:18
cardano    2022-10-24 03:47:38
Name: last_updated_at, dtype: datetime64[ns]


In [46]:
#reformat volume
marketcap_vol_change_df['usd'] = marketcap_vol_change_df['usd'].map("${:,.2f}".format)
marketcap_vol_change_df['usd_24h_vol'] = marketcap_vol_change_df['usd_24h_vol'].map("${:,.2f}".format)
marketcap_vol_change_df['usd_market_cap'] = marketcap_vol_change_df['usd_market_cap'].map("${:,.2f}".format)
marketcap_vol_change_df

,usd,usd_market_cap,usd_24h_vol,usd_24h_change,last_updated_at
polkadot,$5.87,"$6,836,980,256.36","$227,617,028.27",0.130016,2022-10-24 03:47:38
ethereum,"$1,349.23","$162,595,858,967.04","$8,845,903,947.22",2.764293,2022-10-24 03:47:27
solana,$28.62,"$10,256,236,232.39","$703,226,376.76",0.752034,2022-10-24 03:47:29
bitcoin,"$19,391.05","$372,036,670,113.26","$22,870,141,203.42",1.009036,2022-10-24 03:47:18
cardano,$0.36,"$12,497,116,368.33","$372,020,783.69",0.521980,2022-10-24 03:47:38


STOCK

In [68]:
# Tesla Facebook Microsoft Apple Google
symbols = 'meta aapl msft goog tsla' 
# Make asynchronous requests
stock_data = Ticker(symbols, asynchronous=True)
stock_data

In [69]:
# View data available through the 'Summary' tab
summary_details = stock_data.summary_detail


In [81]:
#dataframe and transposed
stock_df=pd.DataFrame(summary_details).T
stock_df.columns


Index(['maxAge', 'priceHint', 'previousClose', 'open', 'dayLow', 'dayHigh',
       'regularMarketPreviousClose', 'regularMarketOpen',
       'regularMarketDayLow', 'regularMarketDayHigh', 'payoutRatio', 'beta',
       'trailingPE', 'forwardPE', 'volume', 'regularMarketVolume',
       'averageVolume', 'averageVolume10days', 'averageDailyVolume10Day',
       'bid', 'ask', 'bidSize', 'askSize', 'marketCap', 'fiftyTwoWeekLow',
       'fiftyTwoWeekHigh', 'priceToSalesTrailing12Months', 'fiftyDayAverage',
       'twoHundredDayAverage', 'trailingAnnualDividendRate',
       'trailingAnnualDividendYield', 'currency', 'fromCurrency', 'toCurrency',
       'lastMarket', 'coinMarketCapLink', 'algorithm', 'tradeable',
       'dividendRate', 'dividendYield', 'exDividendDate',
       'fiveYearAvgDividendYield'],
      dtype='object')

In [82]:
#Desired columns
columns=['open','dayLow', 'dayHigh',
       'regularMarketPreviousClose', 'regularMarketOpen',
       'regularMarketDayLow', 'regularMarketDayHigh','volume', 'regularMarketVolume']
stock_df = stock_df.loc[:, columns].copy()
stock_df

,open,dayLow,dayHigh,regularMarketPreviousClose,regularMarketOpen,regularMarketDayLow,regularMarketDayHigh,volume,regularMarketVolume
goog,98.46,98.23,101.62,100.53,98.46,98.23,101.62,27862280,27862280
aapl,142.87,142.649,147.8473,143.39,142.87,142.649,147.8473,86548609,86548609
meta,126.31,125.45,130.08,131.53,126.31,125.45,130.08,43918643,43918643
tsla,206.415,203.81,214.66,207.28,206.415,203.81,214.66,75713754,75713754
msft,234.74,234.5,242.88,236.15,234.74,234.5,242.88,25235911,25235911
